In [7]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


In [ ]:
import os
original_dir = os.getcwd()
os.chdir(os.path.dirname(os.path.dirname(os.getcwd())))
exec(open('setup/default.py').read())

In [2]:
menu_w_sold_out = pd.read_csv(os.getcwd()+'/data/preproc/main/menu_w_soldout.csv')

In [3]:
menu_w_sold_out

,post_no,day,day_of_week,meal_time,course_no,course,course_kcal,course_protein,course_na,soldout,soldout_time,is_soldout
0,926901171,2023-05-03,Wednesday,lunch,[1],"투움바파스타,미니함박&데미소스,오이피클,탄산음료",956,36,1270.0,[1],NaN,True
1,926901171,2023-05-03,Wednesday,lunch,[2],"닭곰탕,잡곡밥,언양식불고기,쥐어채볶음,고들빼기무침",882,44,1245.0,NaN,NaN,False
2,926901172,2023-05-03,Wednesday,dinner,[1],"육감쫄면,우동국물,후리가케밥(소),갈비만두찜,꼬들단무지",942,58,1466.0,NaN,NaN,False
3,926901172,2023-05-03,Wednesday,dinner,[2],"차돌된장찌개,잡곡밥,한식잡채,고구마순들깨나물,청경채겉절이",631,37,1429.0,NaN,NaN,False
4,926901173,2023-05-04,Thursday,lunch,[1],"<제철식재>매실매실청돈육불고기,잡곡밥,얼큰콩나물국,느타리버섯채소볶음,양념깻잎지",699,30,1251.0,NaN,NaN,False
...,...,...,...,...,...,...,...,...,...,...,...,...
1706,926901166,2023-04-28,Friday,lunch,[1],"열무비빔국수,사골파국,보리밥(소),손만두찜*초간장,오복지무침",944,33,1114.0,NaN,NaN,False
1707,926901166,2023-04-28,Friday,lunch,[2],"돼지국밥*새우젓,잡곡밥,두부양념조림,미나리오징어초무침,브로콜리통마늘볶음",767,42,1409.0,[2],NaN,True
1708,926901167,2023-04-28,Friday,dinner,[1],"소고기숙주덮밥,미소시루,야채튀김&칠리소스,락교무침",883,39,1491.0,NaN,NaN,False
1709,926901169,2023-05-02,Tuesday,lunch,[1],"돈육김치찌개,잡곡밥,도톰동그랑땡전,감자채볶음,풀무원조미김",757,58,1484.0,NaN,NaN,False


In [10]:
X

,day_of_week,meal_time,course_no,course_kcal,course_protein,course_na
2,Wednesday,dinner,[1],942,58,1466.0
3,Wednesday,dinner,[2],631,37,1429.0
6,Thursday,dinner,[1],768,39,1442.0
7,Thursday,dinner,[2],986,55,1226.0
10,Monday,dinner,[1],777,34,1229.0
...,...,...,...,...,...,...
1700,Wednesday,dinner,[1],966,40,1496.0
1701,Wednesday,dinner,[2],851,41,1496.0
1704,Thursday,dinner,[1],889,38,1424.0
1705,Thursday,dinner,[2],842,23,1273.0


In [52]:
df = menu_w_sold_out[menu_w_sold_out.meal_time =='dinner']

# 결측치 처리
df.loc[:,'course_kcal'] = df['course_kcal'].fillna(df['course_kcal'].mean().astype(int))
df.loc[:,'course_protein'] = df['course_protein'].fillna(df['course_protein'].mean().astype(int))
df.loc[:, 'course_na'] = df['course_na'].apply(lambda x: int(x) if pd.notna(x) else np.nan)
df.loc[:,'course_na'] = df['course_na'].fillna(df['course_na'].mean().astype(int))

# 특징과 타겟 변수 설정
X = df.drop(columns=['post_no','day','meal_time','is_soldout', 'course', 'soldout','soldout_time'])
y = df['is_soldout'].astype(int)

# 범주형 변수 인코딩 및 스케일링
numeric_features = ['course_kcal', 'course_protein', 'course_na']
categorical_features = ['day_of_week', 'course_no']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ]
)

/Users/user/anaconda3/envs/first_pytorch/lib/python3.6/site-packages/pandas/core/indexing.py:1743: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(ilocs[0], value)


In [53]:
# 로지스틱 회귀 모델 파이프라인
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', LogisticRegression(max_iter=1000))
])

In [54]:

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


In [56]:
# 모델 훈련
model.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['course_kcal',
                                                   'course_protein',
                                                   'course_na']),
                                                 ('cat', OneHotEncoder(),
                                                  ['day_of_week',
                                                   'course_no'])])),
                ('classifier', LogisticRegression(max_iter=1000))])

In [57]:

# 예측
y_pred = model.predict(X_test)

# 결과 평가
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')
print(classification_report(y_test, y_pred))

Accuracy: 0.8064516129032258
              precision    recall  f1-score   support

           0       0.81      0.99      0.89       202
           1       0.00      0.00      0.00        46

    accuracy                           0.81       248
   macro avg       0.41      0.50      0.45       248
weighted avg       0.66      0.81      0.73       248

